# Machine Learning (Classificação, Regressão, Clustering e Sistema de Recomendação)

### Descriçao

---
1. O objetivo deste notebook é aprender as 4 principais tarefas de ML (Classificação, Regressão, Agrupagamentos, Sistema de Recomendação)
2. Vamos construir um conjunto de dados baseado no universo do Harry Potter
3. O problema consiste em prever, baseado no histórico escolar do aluno, se ele está apto a entrar na casa que ele se candidatou.
---

# Instalação dos pacotes

In [1]:
!pip install pandas numpy scikit-learn Faker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.6 MB/s eta 0:00:00


# Importando as Bibliotecas

In [2]:
# Bibliotecas para manipulação e geração de dados
import pandas as pd  # Manipulação de dados tabulares (DataFrames)
import numpy as np  # Operações com arrays e matrizes numéricas
import random  # Geração de números aleatórios
from faker import Faker  # Geração de dados fictícios (ex: nomes, endereços)

# Bibliotecas do Scikit-Learn para Machine Learning
from sklearn.model_selection import train_test_split  # Divisão dos dados em conjuntos de treino e teste
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder  # Pré-processamento para padronização e codificação de variáveis
from sklearn.compose import ColumnTransformer  # Aplicação de transformações diferentes para colunas específicas

# Métricas para avaliação de modelos
from sklearn.metrics import (
  mean_absolute_error,  # Erro absoluto médio
  mean_squared_error,  # Erro quadrático médio
  r2_score,  # Coeficiente de determinação (R²)
  accuracy_score,  # Acurácia (para classificadores)
  silhouette_score  # Pontuação para avaliação de agrupamentos (clustering)
)

# Modelos de machine learning
from sklearn.ensemble import RandomForestClassifier  # Classificador baseado em floresta de árvores de decisão
from sklearn.linear_model import LogisticRegression  # Modelo de regressão logística
from sklearn.cluster import KMeans  # Algoritmo de agrupamento (clustering)
from sklearn.neighbors import NearestNeighbors  # Algoritmo usado para sistemas de recomendação (vizinhos mais próximos)

# Utilitários
import joblib  # Salvamento e carregamento de modelos treinados
import os  # Manipulação de diretórios e arquivos no sistema operacional

# Construindo o Dataset Faker



In [3]:
def generate(num):  # Recebe um valor numérico que define a quantidade de linhas (alunos) a serem geradas
  fake = Faker()  # Instancia um objeto Faker para gerar dados fictícios
  data = []  # Lista onde serão armazenados os dados gerados

  for i in range(num):  # Loop para gerar os dados de cada aluno
    aluno = fake.name()  # Gera um nome fictício
    idade = random.randint(11, 39)  # Gera uma idade aleatória entre 11 e 39 anos
    casas = random.choice(['Grifinória', 'Sonserina', 'Corvinal', 'Lufa-lufa'])  # Escolhe uma casa aleatoriamente
    ano_de_ingresso = random.randint(2024 - idade, 2024)  # Define o ano de ingresso com base na idade

    nota_1 = round(random.uniform(0, 100), 2)  # Gera uma nota aleatória entre 0 e 100 (ensino fundamental)
    # Se nota_1 for menor que 50, nota_2 (ensino médio) será 0. Caso contrário, gera uma nova nota aleatória
    nota_2 = 0 if nota_1 < 50 else round(random.uniform(0, 100), 2)
    # Se nota_2 for 0, a nota final será igual à nota_1; caso contrário, é a média entre as duas
    nota_final = nota_1 if nota_2 == 0 else round((nota_1 + nota_2) / 2, 2)
    # Define a classificação final com base na nota final
    label_alvo = 'reprovado' if nota_final < 50 else 'aprovado'

    # Cria o dicionário com os dados do aluno
    row = {
      'aluno': aluno,
      'idade': idade,
      'casas': casas,
      'ano_de_ingresso': ano_de_ingresso,
      'ensino_fundamental': nota_1,
      'ensino_medio': nota_2,
      'nota_final': nota_final,
      'label_alvo': label_alvo
    }

    data.append(row)  # Adiciona o dicionário à lista

  df = pd.DataFrame(data)  # Converte a lista de dicionários em um DataFrame
  return df  # Retorna o DataFrame gerado

# Geração de um DataFrame com 1000 alunos
df = generate(1000)
df  # Exibe o DataFrame

,aluno,idade,casas,ano_de_ingresso,ensino_fundamental,ensino_medio,nota_final,label_alvo
0,Tamara Oconnell,13,Sonserina,2014,44.69,0.00,44.69,reprovado
1,Melissa Watson,30,Lufa-lufa,2001,96.72,40.54,68.63,aprovado
2,Joshua Macdonald,18,Grifinória,2019,90.26,17.81,54.04,aprovado
3,Erin Rogers,37,Corvinal,1996,22.65,0.00,22.65,reprovado
4,Michael Carlson,24,Corvinal,2024,11.14,0.00,11.14,reprovado
...,...,...,...,...,...,...,...,...
995,Dwayne Silva,37,Sonserina,1990,84.61,93.21,88.91,aprovado
996,Brenda Velazquez,28,Grifinória,2011,61.11,66.77,63.94,aprovado
997,Jason Santos,31,Grifinória,2009,31.82,0.00,31.82,reprovado
998,Justin Ward,31,Lufa-lufa,2013,53.56,87.80,70.68,aprovado


# Classificação

#### **Objetivo**
1. Na classificação, nosso objetivo é prever ou descrever uma variavel categorica (string)
2. Vamos usar a coluna catégorica [label_alvo] para nosso estudo.

In [4]:
# Cria uma cópia do DataFrame original removendo a coluna 'aluno',
# já que nomes não são relevantes para análise e podem ser considerados dados sensíveis
df_new = df.drop(['aluno'], axis=1)

df_new  # Exibe o novo DataFrame sem a coluna 'aluno'

,idade,casas,ano_de_ingresso,ensino_fundamental,ensino_medio,nota_final,label_alvo
0,13,Sonserina,2014,44.69,0.00,44.69,reprovado
1,30,Lufa-lufa,2001,96.72,40.54,68.63,aprovado
2,18,Grifinória,2019,90.26,17.81,54.04,aprovado
3,37,Corvinal,1996,22.65,0.00,22.65,reprovado
4,24,Corvinal,2024,11.14,0.00,11.14,reprovado
...,...,...,...,...,...,...,...
995,37,Sonserina,1990,84.61,93.21,88.91,aprovado
996,28,Grifinória,2011,61.11,66.77,63.94,aprovado
997,31,Grifinória,2009,31.82,0.00,31.82,reprovado
998,31,Lufa-lufa,2013,53.56,87.80,70.68,aprovado


In [5]:
# Separação das variáveis independentes (features) e da variável dependente (rótulo/target)

X = df_new.iloc[:, :-1]  # Seleciona todas as colunas, exceto a última ('label_alvo'), como features
y = df_new['label_alvo']  # Define a coluna 'label_alvo' como variável alvo (target)

In [6]:
# Codificação de variáveis categóricas para formato numérico

label_encoder = LabelEncoder()  # Instancia o codificador de rótulos

# Codifica a coluna 'casas' (variável categórica) para valores numéricos
X['casas'] = label_encoder.fit_transform(X['casas'])

# Codifica os valores da variável alvo 'label_alvo' (aprovado/reprovado) em números (0 e 1)
y = label_encoder.fit_transform(y)

In [7]:
X.head()  # Exibe as primeiras linhas do DataFrame X, que agora contém variáveis numéricas

,idade,casas,ano_de_ingresso,ensino_fundamental,ensino_medio,nota_final
0,13,3,2014,44.69,0.00,44.69
1,30,2,2001,96.72,40.54,68.63
2,18,1,2019,90.26,17.81,54.04
3,37,0,1996,22.65,0.00,22.65
4,24,0,2024,11.14,0.00,11.14


In [8]:
# As categorias transformadas em números podem induzir o modelo ao erro,
# pois ele pode interpretar valores maiores como superiores — o que não é verdade para variáveis categóricas nominais.
# Para evitar isso, usamos codificação one-hot.

# --- Método 1: Utilizando get_dummies do pandas ---
# Converte a coluna 'casas' em variáveis dummies (colunas binárias) e força o tipo int para consistência
f = pd.get_dummies(X, columns=['casas']).astype(int)


# --- Método 2: Utilizando OneHotEncoder do Scikit-learn ---
# Usamos ColumnTransformer para aplicar o OneHotEncoder apenas à coluna 'casas' (índice 1),
# mantendo as demais colunas inalteradas (remainder='passthrough')
ct = ColumnTransformer(
  [("Casas", OneHotEncoder(), [1])],  # Nome, método e índice da coluna a codificar
  remainder='passthrough'  # Mantém as outras colunas sem alterações
)

# Aplica a transformação
o = ct.fit_transform(X)

In [9]:
# Exibe as primeiras 5 linhas do DataFrame 'f' (versão de X com codificação one-hot via get_dummies)
f.head()

,idade,ano_de_ingresso,ensino_fundamental,ensino_medio,nota_final,casas_0,casas_1,casas_2,casas_3
0,13,2014,44,0,44,0,0,0,1
1,30,2001,96,40,68,0,0,1,0
2,18,2019,90,17,54,0,1,0,0
3,37,1996,22,0,22,1,0,0,0
4,24,2024,11,0,11,1,0,0,0


In [11]:
# Divisão dos dados em conjuntos de treino e teste

X_train, X_test, y_train, y_test = train_test_split(
  o,              # Features codificadas com OneHotEncoder
  y,              # Labels codificados (0 = reprovado, 1 = aprovado)
  test_size=0.2,  # 20% dos dados serão usados para teste
  random_state=42 # Semente para garantir reprodutibilidade dos resultados
)

In [12]:
# Inicialização e treinamento do modelo de classificação usando Random Forest

forest_classifier = RandomForestClassifier(
  n_estimators=20,  # Número de árvores na floresta (quanto maior, mais robusto, mas mais custoso)
  random_state=42   # Semente para reprodutibilidade
)

# Treinamento do modelo com os dados de treino
forest_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=20, random_state=42)

In [13]:
# Exibe o tamanho (quantidade de amostras) dos conjuntos de treino e teste

print(f"O tamanho do treino: {len(X_train)}")  # Quantidade de amostras usadas para treinar o modelo
print(f"O tamanho do teste: {len(X_test)}")    # Quantidade de amostras usadas para testar o modelo

O tamanho do treino: 800
O tamanho do teste: 200


In [16]:
from sklearn.metrics import (
  confusion_matrix,
  classification_report,
  roc_auc_score
)

# Previsão com os dados de teste
y_pred = forest_classifier.predict(X_test)

# Acurácia: proporção de acertos
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo: {accuracy:.2f}')

# Matriz de confusão: mostra os acertos e erros de classificação
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

# Relatório de classificação: mostra precisão, recall, f1-score e suporte
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

# ROC AUC Score: área sob a curva ROC (apenas para classificação binária)
# Verifica se o problema é binário antes de calcular
if len(np.unique(y_test)) == 2:
  y_prob = forest_classifier.predict_proba(X_test)[:, 1]  # Probabilidades da classe positiva
  auc = roc_auc_score(y_test, y_prob)
  print(f"ROC AUC Score: {auc:.2f}")

Acurácia do modelo: 1.00

Matriz de Confusão:
[[ 71   0]
 [  0 129]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        71
           1       1.00      1.00      1.00       129

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200

ROC AUC Score: 1.00


#### Salvando o modelo

In [19]:
# Criação de uma pasta para armazenar o modelo treinado
folder = 'classificacao'

os.mkdir(folder)  # Cria a pasta 'classificacao' (certifique-se de que ela ainda não exista)

# Salva o modelo Random Forest treinado em um arquivo .pkl usando joblib
with open(f'{folder}/aprovacao_random_forest.pkl', 'wb') as f:
    joblib.dump(forest_classifier, f)

### Reutilizando o modelo

In [22]:
# Carrega o modelo Random Forest salvo anteriormente
rf_classifier = joblib.load(f'{folder}/aprovacao_random_forest.pkl')

In [31]:
# Testando o modelo com um novo aluno

nome_candidado = 'Daniel Ferreira'  # Nome fictício do novo aluno
idade = 35
casa_pretendida = 'Lufa-lufa'
ano_de_ingresso = 2024
ensino_fundamental = 58.9
ensino_medio = 10.0
casas_0 = 1
casas_1 = 0
casas_2 = 0
casas_3 = 0

# Calcula a média das notas
nota_final = (ensino_fundamental + ensino_medio) / 2

# Cria o vetor com os dados numéricos (sem incluir 'casas' ainda)
X_novo = [[idade, ano_de_ingresso, ensino_fundamental, ensino_medio, nota_final, casas_0, casas_1, casas_2, casas_3]]

# Previsão com o modelo carregado
y_pred = rf_classifier.predict(X_novo)

# Decodifica o resultado numérico de volta para "aprovado" ou "reprovado"
resultado = label_encoder.inverse_transform(y_pred)[0]

# Gera a resposta final formatada
resposta = f"O aluno {nome_candidado} pretendido a casa {casa_pretendida} foi {resultado.upper()}"

In [32]:
resposta

'O aluno Daniel Ferreira pretendido a casa Lufa-lufa foi REPROVADO'

# Regressão

#### **Objetivo**
1. Em regressão eu tento prever ou descrever uma categoria númerica (int)
2. Neste caso iremos criar uma regressão para podermos prever, com base nas notas final quais sao as chances do aluno de entrar na casa pretendida.




In [33]:
# Gerando um novo conjunto de dados fictício com 1000 alunos
# A função generate() retorna um DataFrame com dados simulados para:
# - Nome do aluno (falso)
# - Idade
# - Casa (Grifinória, Sonserina, Corvinal, Lufa-lufa)
# - Ano de ingresso
# - Nota do ensino fundamental
# - Nota do ensino médio (ou 0 se reprovado na etapa anterior)
# - Nota final (média entre as duas)
# - Label de aprovação ou reprovação

df = generate(1000)  # Gera 1000 registros
df  # Exibe o DataFrame gerado

,aluno,idade,casas,ano_de_ingresso,ensino_fundamental,ensino_medio,nota_final,label_alvo
0,Mark Goodman,32,Lufa-lufa,2000,47.07,0.00,47.07,reprovado
1,Dr. Marc Carter,14,Sonserina,2021,33.44,0.00,33.44,reprovado
2,Daniel Summers,20,Corvinal,2013,77.99,7.00,42.49,reprovado
3,Elizabeth Harrington,28,Grifinória,2005,74.18,73.19,73.69,aprovado
4,David Morgan,11,Lufa-lufa,2021,63.49,64.05,63.77,aprovado
...,...,...,...,...,...,...,...,...
995,Kevin Lopez,35,Grifinória,2017,21.29,0.00,21.29,reprovado
996,Corey Carr,21,Corvinal,2020,43.50,0.00,43.50,reprovado
997,Sarah Martinez,25,Lufa-lufa,2013,1.98,0.00,1.98,reprovado
998,Richard Valentine,29,Sonserina,2006,0.11,0.00,0.11,reprovado


In [34]:
# Transformando variáveis categóricas ('label_alvo' e 'casas') em valores numéricos
# Isso é necessário para que algoritmos de machine learning consigam processar os dados

label_encoder = LabelEncoder()  # Instancia o codificador de rótulos

# Converte os valores 'aprovado'/'reprovado' da coluna 'label_alvo' em 0 e 1
df['label_alvo'] = label_encoder.fit_transform(df['label_alvo'])

# Converte os nomes das casas (ex: 'Grifinória', 'Sonserina') em valores numéricos
df['casas'] = label_encoder.fit_transform(df['casas'])

In [35]:
# Separando os dados para um modelo supervisionado (classificação)
# X contém as features (variáveis independentes) que serão usadas para prever a aprovação
# y contém o rótulo (variável dependente), que indica se o aluno foi aprovado ou reprovado

X = df[['nota_final', 'casas']]  # Seleciona apenas as colunas relevantes como entrada do modelo
y = df['label_alvo']             # Coluna alvo: 0 (reprovado) ou 1 (aprovado)

In [36]:
# Dividindo o conjunto de dados em treino e teste
# - 80% dos dados serão usados para treinar o modelo (X_train, y_train)
# - 20% dos dados serão usados para testar o modelo (X_test, y_test)
# - random_state garante que a divisão seja sempre a mesma ao rodar o código novamente (reprodutibilidade)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
# Criando e treinando um modelo de Regressão Logística
# Esse modelo será usado para prever se o aluno será aprovado ou reprovado
# com base nas variáveis: nota_final e casa (codificada numericamente)

regression_model = LogisticRegression()         # Instancia o modelo de regressão logística
regression_model.fit(X_train, y_train)          # Treina o modelo com os dados de treino

LogisticRegression()

In [38]:
from sklearn.metrics import classification_report

# Fazendo previsões com os dados de teste
y_pred = regression_model.predict(X_test)

# Gerando o relatório com precisão, recall e f1-score
relatorio = classification_report(y_test, y_pred, target_names=['Reprovado', 'Aprovado'])

print("Relatório de desempenho do modelo:\n")
print(relatorio)

Relatório de desempenho do modelo:

              precision    recall  f1-score   support

   Reprovado       1.00      0.98      0.99        66
    Aprovado       0.99      1.00      1.00       134

    accuracy                           0.99       200
   macro avg       1.00      0.99      0.99       200
weighted avg       1.00      0.99      0.99       200



#### Salvando o modelo

In [39]:
# Salvando o modelo de regressão logística treinado
# Isso permite reutilizar o modelo posteriormente sem precisar reentreinar

folder = 'regressao'  # Nome da pasta onde o modelo será salvo

os.mkdir(folder)  # Cria a pasta (se ainda não existir)

# Abre o arquivo em modo escrita binária e salva o modelo usando joblib
with open(f'{folder}/regression_logistic.pkl', 'wb') as f:
    joblib.dump(regression_model, f)

### Reutilizando o modelo

In [40]:
# Carregando o modelo de regressão logística salvo anteriormente
# Isso evita a necessidade de treinar o modelo novamente

lg = joblib.load(f'{folder}/regression_logistic.pkl')  # 'lg' é a variável que armazenará o modelo carregado

In [41]:
# Simulação de previsão com o modelo de regressão logística

aluno = 'Daniel Ferreira'
casa_pretendida = 'Grifinória'
nota_final = 50

# Transformando a casa (categórica) para valor numérico com o mesmo LabelEncoder usado no treino
transfor_casa = label_encoder.transform([casa_pretendida])[0]

# Preparando os dados de entrada (deve seguir o mesmo formato usado no treino)
entrada = [[nota_final, transfor_casa]]

# Predict_proba retorna a probabilidade de cada classe (Reprovado, Aprovado)
# Ex: [0.3, 0.7] → 30% de chance de reprovar, 70% de chance de aprovar
resultado = lg.predict_proba(entrada)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [42]:
# Exibindo o resultado da previsão de forma legível

print(f"Resultado final: {resultado}")  # Mostra as probabilidades de [reprovado, aprovado]
print(f"O aluno {aluno} tem {round(resultado[0][0], 2) * 100}% de chance de entrar na {casa_pretendida}")

Resultado final: [[0.30328083 0.69671917]]
O aluno Daniel Ferreira tem 30.0% de chance de entrar na Grifinória


# Agrupamentos (clustering)

#### **Objetivo**
1. Em agrupamentos eu não tento prever uma categoria especifica, tento agrupar os dados de acordo com caracteristicas em comum.
2. Neste nosso caso vamos criar um cluster para entender como os alunos realmente estão dividos entre si, pelas notas de ensino, pelo ano de ingresso, ou pela casa que se candidataram.
3. [Documentação dos algorítimos](https://scikit-learn.org/stable/modules/clustering.html#clustering)

In [ ]:
df = generate(100000)
df

In [ ]:
X = df.drop(['aluno'], axis=1)  # Remover as colunas 'aluno' e 'label_alvo'
X = X[X['label_alvo'] == 'aprovado'] # Filtrando apenas os aprovados
X = X.drop(['label_alvo'], axis=1) #agora que sei que meu dataset só tem gente aprovada, removo a label
X.shape

In [ ]:
# Converter a coluna 'casas' em valores numéricos
le = LabelEncoder()
X['casas'] = le.fit_transform(X['casas'])


In [ ]:
# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Criar e treinar o modelo KMeans
km = KMeans(n_clusters=4, random_state=42)
clusters = km.fit_predict(X_scaled)

In [ ]:
# Adicionar os clusters ao DataFrame
df['grupo'] = clusters

In [ ]:
df

In [ ]:
df['grupo'].unique()

In [ ]:
df.loc[df['grupo'] == 0]

In [ ]:
# Calcular a inércia
inertia = km.inertia_
# Calcular o coeficiente de silhueta
silhouette = silhouette_score(X_scaled, clusters)

In [ ]:
print(f'Inércia: {inertia}')
print(f'Coeficiente de Silhueta: {silhouette}')

### Salvando o modelo

In [ ]:
folder = 'clustering'

os.mkdir(folder)
with open(f'{folder}/harrypotter_clustering.pkl', 'wb') as f:
    joblib.dump(km, f)


### Testando com novos dados

In [ ]:
#importando o modelo
km = joblib.load(f'{folder}/harrypotter_clustering.pkl')

In [ ]:
df_new = generate(10000) #gerando uma base de dados maior
X = df_new.drop(['aluno'], axis=1)  # Remover as colunas 'aluno' e 'label_alvo'

#o processo de pre processamento
le = LabelEncoder()
X['casas'] = le.fit_transform(X['casas'])
X['label_alvo'] = le.fit_transform(X['label_alvo'])

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Criar e treinar o modelo KMeans
km = KMeans(n_clusters=4, random_state=42)
clusters = km.fit_predict(X_scaled)

df_new['grupo'] = clusters

inertia = km.inertia_
# Calcular o coeficiente de silhueta
silhouette = silhouette_score(X_scaled, clusters)

In [ ]:
print(f'Inércia: {inertia}')
print(f'Coeficiente de Silhueta: {silhouette}')

# Sistema de Recomendação

#### **Objetivo**
1.  Sistemas de recomendação são frequentemente implementados usando técnicas de aprendizado supervisionado, não supervisionado ou baseados em conteúdo.
2. Vamos recomendar a casa para o aluno, baseado na sua idade.
3. [Documentação dos algorítimos](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors)



In [ ]:
df

In [ ]:
#criando nosso dataframe

df = generate(1000)
df

In [ ]:
df = df.loc[df['label_alvo'] == 'aprovado'] #do nosso dataset, vamos pegar apenas quem foi aprovado. Nao faz sentido recomendar para quem foi reprovado

In [ ]:
#selecionar apenas as variaveis que queremos

X = df[['idade', 'ano_de_ingresso']]
y = df['casas']

In [ ]:
# Transformando casas para um valor numerico

le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
#Treinando o sistema
vizinho = NearestNeighbors(algorithm = 'brute') #Algoritimo é um parametro usado para buscar os vizinhos de forma mais bruta
vizinho.fit(X, y)

#### Salvando o modelo

In [ ]:
folder = 'recomendacao'

os.mkdir(folder)
with open(f'{folder}/harrypotter_recomendacao.pkl', 'wb') as f:
    joblib.dump(vizinho, f)

#### Importando o modelo

In [ ]:
#importando o modelo
viz = joblib.load(f'{folder}/harrypotter_recomendacao.pkl')

####testando

In [ ]:
aluno = 'Maycon Batestin'
idade = 43
ano_de_ingresso = 2024

In [ ]:
# Criar o novo aluno
novo_aluno = pd.DataFrame({'idade': [idade], 'ano_de_ingresso': [ano_de_ingresso]})

In [ ]:
# Criar o modelo Nearest Neighbors
viz.fit(X, y)

In [ ]:
distancias, indices_vizinhos = vizinho.kneighbors(novo_aluno) # Encontrar os vizinhos mais próximos do novo aluno


In [ ]:
# Obter as casas dos vizinhos mais próximos
casas_vizinhos = df.iloc[indices_vizinhos[0]]['casas'].values


In [ ]:
# Encontrar a casa mais comum entre os vizinhos
casa_recomendada = pd.Series(casas_vizinhos).mode()[0] #aqui pegamos da serie casa_vizinhos que estabelecemos acima, e puxamos o primeiro item(0) para a variavel casa_recomendada


In [ ]:

print(f"Recomendamos para o {aluno} a casa {casa_recomendada.upper()}")